# **Forecast casas y departamentos**

In [2]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

ImportError: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Tipo':'tipo'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 
    segment_df = globals()[nombre_df][['tipo']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [5]:
import re
# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Extraer mes y año usando expresiones regulares
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()
        ano = int(match.group(2))
        return mes, ano
    return None, None

# Crear un DataFrame vacío para almacenar los resultados
df_resultados = pd.DataFrame(columns=['tipo', 'conteo', 'mes', 'año'])

# Iterar sobre los nombres de los DataFrames y asignar las columnas 'mes' y 'año'
for nombre_df in nombres_df_filtrados:
    df = globals()[nombre_df]
     # Cambiar los valores de 'tipo' para que sean 'casa' o 'departamento'
    df['tipo'] = df['tipo'].apply(lambda x: 'casa' if 'casa' in x.lower() else 'departamento')
    mes, ano = extraer_mes_y_ano(nombre_df)
    if mes is not None and ano is not None:
        conteo = df['tipo'].value_counts().reset_index()
        conteo.columns = ['tipo', 'conteo']
        conteo['mes'] = mes
        conteo['año'] = ano
        # Cambiar los valores de 'tipo' para que sean 'casa' o 'departamento'
        #conteo['tipo'] = conteo['tipo'].apply(lambda x: 'casa' if 'casa' in x.lower() else 'departamento')
        df_resultados = pd.concat([df_resultados, conteo], ignore_index=True)

df_resultados


,tipo,conteo,mes,año
0,casa,1492,abril,2024
1,departamento,55,abril,2024
2,casa,1333,agosto,2024
3,departamento,73,agosto,2024
4,casa,1667,febrero,2024
5,departamento,229,febrero,2024
6,casa,1700,julio,2024
7,departamento,63,julio,2024
8,casa,1524,junio,2024
9,departamento,101,junio,2024


PROMEDIO DE AÑOS

In [6]:
promedio_resultados = df_resultados.groupby(['tipo','año'])['conteo'].mean().reset_index()
promedio_resultados.columns = ['tipo','año','promedio']
promedio_resultados['promedio'] = promedio_resultados['promedio'].round(2)
promedio_resultados

,tipo,año,promedio
0,casa,2022,1714.00
1,casa,2023,1213.20
2,casa,2024,1571.43
3,departamento,2022,90.50
4,departamento,2023,58.25
5,departamento,2024,92.86


### Predicción hasta 2027, a partir de diferencia porcentual entre 2023 y 2024

In [7]:
promedio_2023 = promedio_resultados[promedio_resultados['año'] == 2023].set_index('tipo')['promedio']
promedio_2024 = promedio_resultados[promedio_resultados['año'] == 2024].set_index('tipo')['promedio']
diferencia_porcentual = round(((promedio_2024 - promedio_2023) / promedio_2023) * 100, 2)

# Proyecciones a los siguientes años, Comenzando con 2024
predicciones = pd.DataFrame(index=promedio_2024.index)
predicciones[2024] = promedio_2024

# Proyectar los siguientes años aplicando la diferencia porcentual acumulativa
for año in range(2025, 2028):
    # Aplicar el crecimiento porcentual sobre el valor del año anterior
    predicciones[año] = round(predicciones[año-1] * (1 + diferencia_porcentual / 100), 2)
df_predicciones = pd.DataFrame(predicciones)
df_predicciones = df_predicciones.reset_index()
predict_pachuca = pd.melt(df_predicciones, id_vars='tipo', var_name='año', value_name='promedio')
predict_pachuca

,tipo,año,promedio
0,casa,2024,1571.43
1,departamento,2024,92.86
2,casa,2025,2035.47
3,departamento,2025,148.04
4,casa,2026,2636.54
5,departamento,2026,236.01
6,casa,2027,3415.11
7,departamento,2027,376.25


In [8]:
df_concatenado = pd.concat([promedio_resultados, predict_pachuca], ignore_index=True)
df_concatenado['promedio'] = df_concatenado['promedio'].round()
df_concatenado

,tipo,año,promedio
0,casa,2022,1714.0
1,casa,2023,1213.0
2,casa,2024,1571.0
3,departamento,2022,90.0
4,departamento,2023,58.0
5,departamento,2024,93.0
6,casa,2024,1571.0
7,departamento,2024,93.0
8,casa,2025,2035.0
9,departamento,2025,148.0


In [9]:
# Datos
años = df_concatenado['año']
tipos = df_concatenado['tipo']
promedios = df_concatenado['promedio']

fig = go.Figure()
# Agregar líneas para cada tipo
for tipo in tipos.unique():
    df_tipo = df_concatenado[df_concatenado['tipo'] == tipo]
    fig.add_trace(go.Scatter(
        x=df_tipo['año'],
        y=df_tipo['promedio'],
        mode='markers+lines',
        name=tipo
    ))

fig.update_layout(
    title='Proyección demanda casas/departamentos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Modificar las etiquetas del eje x
fig.update_xaxes(
    tickvals=años,  # Establecer las posiciones de las marcas del eje x
    ticktext=años,  # Establecer las etiquetas del eje x
    title='Año'
)
# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_tipo = df_concatenado[df_concatenado['tipo'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_tipo.iloc[i]["promedio"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
#def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
   # if not os.path.exists(carpeta):
  #      os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
 #   pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
#guardar_grafico_como_html(fig, 'g_scatt_casasdepart_proyecdemanda_pachuca', carpeta='graficas')

fig.show()


## *Proyeccion anual oferta casas/departamentos*

In [10]:
#proyeccion_anual_pachuca = []
#
#for tipo in df_concatenado['tipo'].unique():
#    for año in df_concatenado[df_concatenado['tipo'] == tipo]['año'].unique():
#        if año > min(df_concatenado['año']):
#            promedio_actual = df_concatenado[(df_concatenado['tipo'] == tipo) & (df_concatenado['año'] == año)]['promedio'].values[0]
#            promedio_anterior = df_concatenado[(df_concatenado['tipo'] == tipo) & (df_concatenado['año'] == año - 1)]['promedio'].values[0]
#            proyeccion = ((promedio_actual - promedio_anterior) / promedio_anterior) * 100
#            proyeccion_anual_pachuca.append([tipo, año, proyeccion])
#df_proyeccion_anual_pachuca = pd.DataFrame(proyeccion_anual_pachuca, columns=['tipo', 'año', 'proyeccion'])
#df_proyeccion_anual_pachuca['proyeccion'] = df_proyeccion_anual_pachuca['proyeccion'].round(2)
#df_proyeccion_anual_pachuca

In [11]:
proyeccion_anual_pachuca = []

for tipo in df_concatenado['tipo'].unique():
    for año in df_concatenado[df_concatenado['tipo'] == tipo]['año'].unique():
        if año > min(df_concatenado['año']):
            promedio_actual = df_concatenado[(df_concatenado['tipo'] == tipo) & (df_concatenado['año'] == año)]['promedio'].values[0]
            promedio_anterior = df_concatenado[(df_concatenado['tipo'] == tipo) & (df_concatenado['año'] == año - 1)]['promedio'].values[0]
            proyeccion = ((promedio_actual - promedio_anterior) / promedio_anterior) * 100
            proyeccion_anual_pachuca.append([tipo, año, proyeccion])
df_proyeccion_anual_pachuca = pd.DataFrame(proyeccion_anual_pachuca, columns=['tipo', 'año', 'proyeccion'])
df_proyeccion_anual_pachuca['proyeccion'] = df_proyeccion_anual_pachuca['proyeccion'].round(2)
df_proyeccion_anual_pachuca['proyeccion'] = df_proyeccion_anual_pachuca['proyeccion'].apply(lambda x: x * 10)
#df_proyeccion_anual_pachuca['proyeccion'] = df_proyeccion_anual_pachuca['proyeccion'].apply(lambda x: round(x, 2))
df_proyeccion_anual_pachuca['proyeccion'] = df_proyeccion_anual_pachuca['proyeccion'].round(2)

df_proyeccion_anual_pachuca

,tipo,año,proyeccion
0,casa,2023,-292.3
1,casa,2024,295.1
2,casa,2025,295.4
3,casa,2026,295.8
4,casa,2027,295.0
5,departamento,2023,-355.6
6,departamento,2024,603.4
7,departamento,2025,591.4
8,departamento,2026,594.6
9,departamento,2027,593.2


In [12]:
años = df_proyeccion_anual_pachuca['año']
tipos = df_proyeccion_anual_pachuca['tipo']
proyecciones = df_proyeccion_anual_pachuca['proyeccion']

fig = go.Figure()
# Agregar líneas para cada tipo
for tipo in tipos.unique():
    df_tipo = df_proyeccion_anual_pachuca[df_proyeccion_anual_pachuca['tipo'] == tipo]
    fig.add_trace(go.Scatter(
        x=df_tipo['año'],
        y=df_tipo['proyeccion'],
        mode='markers+lines',
        name=tipo
    ))
fig.update_layout(
    title='Proyeccion anual oferta casas/departamentos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Modificar las etiquetas del eje x
fig.update_xaxes(
    tickvals=años,  # Establecer las posiciones de las marcas del eje x
    ticktext=años,  # Establecer las etiquetas del eje x
    title='Año'
)
# Formatear las proyecciones como porcentaje con dos decimales y comas para leer las cantidades como 100,00.00
#fig.update_yaxes(tickformat=".2%,")
fig.update_yaxes(tickformat=".2f", title='Proyección')


# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_tipo = df_proyeccion_anual_pachuca[df_proyeccion_anual_pachuca['tipo'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{point:.2%}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

#fig.show()

In [14]:
# Crear figura
fig = go.Figure()

# Agregar líneas para cada tipo
for tipo in tipos.unique():
    df_tipo = df_proyeccion_anual_pachuca[df_proyeccion_anual_pachuca['tipo'] == tipo]
    fig.add_trace(go.Scatter(
        x=df_tipo['año'],
        y=df_tipo['proyeccion'],
        mode='markers+lines',
        name=tipo
    ))

# Configuración del layout del gráfico
fig.update_layout(
    #title='Proyeccion anual de oferta casas/departamentos',
    yaxis=dict(gridcolor='#dddcda'),  # Color del grid
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',     # Fondo del gráfico
)

# Modificar las etiquetas del eje X
fig.update_xaxes(
    tickvals=años,  # Posiciones de las marcas del eje X
    ticktext=años,  # Etiquetas del eje X
    title='Año'
)

# Cambiar el formato del eje Y para mostrar números grandes con comas
fig.update_yaxes(tickformat=',.2f', title='Proyección')

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_tipo = df_proyeccion_anual_pachuca[df_proyeccion_anual_pachuca['tipo'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{point:,.2f}%',  # Mostrar el valor con comas y dos decimales
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
# Guardar el gráfico como archivo HTML
guardar_grafico_como_html(fig, 'g_scatt_casasdepa_proyecanualoferta', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()